<a href="https://colab.research.google.com/github/Srividya023/Smart-OCR-Translator/blob/main/Smart_OCR_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**All the necessary libraries are installed**

In [ ]:
!pip -q install pillow opencv-python pymupdf reportlab pytesseract transformers sentencepiece torch
!apt-get -q -y install tesseract-ocr tesseract-ocr-spa tesseract-ocr-fra tesseract-ocr-deu tesseract-ocr-ita tesseract-ocr-por

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.4 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  tesseract-ocr-deu tesseract-ocr-fra tesseract-ocr-ita tesseract-ocr-por
  tesseract-ocr-spa
0 upgraded, 5 newly installed, 0 to remove and 41 not upgraded.
Need to get 4,146 kB of archives.
After this operation, 9,709 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-deu all 1:4.00~git30-7274cfa-1.1 [744 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-fra all 1:4.00~git30-7274cfa-1.1 [527 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-ita all 1:4.00~git30-7274cfa-1.1 [1,067 kB]
Get:4 http://archive.ubuntu.com/ubuntu ja

**Install basic dejavu fonts so that it can identify which language we uploaded**

In [ ]:
!apt-get install -y fonts-dejavu-core

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-dejavu-core
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 1,041 kB of archives.
After this operation, 3,025 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Fetched 1,041 kB in 2s (548 kB/s)
Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 121723 files and directories currently installed.)
Preparing to unpack .../fonts-dejavu-core_2.37-2build1_all.deb ...
Unpacking fonts-dejavu-core (2.37-2build1) ...
Setting up fonts-dejavu-core (2.37-2build1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


**Upload the doc or image you have (other than english language)**

In [ ]:
from google.colab import files
uploaded = files.upload()          # choose image or pdf(other than english language)
SRC_PATH = list(uploaded.keys())[0]
SRC_PATH

Saving French - SO.pdf to French - SO.pdf


'French - SO.pdf'

**Load pages → Run OCR → Group words into lines → Detect language → Load translator → Translate lines → Draw translated text on pages → Save final PDF**

In [ ]:
# -*- coding: utf-8 -*-
import os, io, re, math
import fitz                         # PyMuPDF
import pytesseract
from PIL import Image, ImageDraw, ImageFont
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

# -------- Settings --------
OCR_DPI = 300
WHITEBOX = True          # cover original text with white rectangles
FONT_SIZE = 14           # try 12–18; adjust per document
TESS_LANGS = "eng+fra+spa+deu+ita+por"  # OCR languages to use initially
OUT_PATH = "Translated_EN_PreservedLayout.pdf"

# ---------- Helpers ----------
def is_pdf(path: str) -> bool:
    return path.lower().endswith(".pdf")

def load_image_or_pdf_pages(path: str, dpi: int = OCR_DPI):
    """Return list of PIL images (RGB) for each page, plus a flag if rasterized from PDF."""
    images = []
    if is_pdf(path):
        doc = fitz.open(path)
        for page in doc:
            pix = page.get_pixmap(dpi=dpi, alpha=False)
            img = Image.open(io.BytesIO(pix.tobytes("png"))).convert("RGB")
            images.append(img)
    else:
        # Single image
        img = Image.open(path).convert("RGB")
        images.append(img)
    return images

def ocr_words_with_boxes(pil_img: Image.Image, tess_langs=TESS_LANGS):
    """Return list of word dicts with text and bbox; then group into lines."""
    data = pytesseract.image_to_data(pil_img, lang=tess_langs, output_type=pytesseract.Output.DICT)
    words = []
    for i in range(len(data["text"])):
        txt = (data["text"][i] or "").strip()
        if not txt:
            continue
        conf = int(data.get("conf", ["-1"])[i]) if data.get("conf") else -1
        x, y, w, h = data["left"][i], data["top"][i], data["width"][i], data["height"][i]
        words.append({"text": txt, "bbox": (x, y, w, h), "conf": conf})
    # group into lines by y proximity
    words.sort(key=lambda w: (w["bbox"][1], w["bbox"][0]))
    lines = []
    current = []
    prev_y = None
    for w in words:
        y = w["bbox"][1]
        if prev_y is None or abs(y - prev_y) <= 10:   # 10px vertical tolerance
            current.append(w)
        else:
            if current:
                lines.append(current[:])
            current = [w]
        prev_y = y
    if current:
        lines.append(current)
    # convert to [(text, (x,y,w,h))]
    line_items = []
    for line in lines:
        if not line:
            continue
        # skip noisy lines (mostly digits) if desired
        text_line = " ".join([w["text"] for w in line]).strip()
        if not text_line:
            continue
        if re.fullmatch(r"[0-9\W]{3,}", text_line):  # mostly numbers/symbols
            continue
        xs = [w["bbox"][0] for w in line]
        ys = [w["bbox"][1] for w in line]
        xe = [w["bbox"][0] + w["bbox"][2] for w in line]
        ye = [w["bbox"][1] + w["bbox"][3] for w in line]
        x1, y1, x2, y2 = min(xs), min(ys), max(xe), max(ye)
        line_items.append((text_line, (x1, y1, x2 - x1, y2 - y1)))
    return line_items

# ---------- Language detect ----------
_langid = None
def detect_lang(text):
    global _langid
    sample = (text or "")[:1500]
    if not sample.strip():
        return "en"
    if _langid is None:
        _langid = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection", truncation=True)
    res = _langid(sample)[0]["label"].lower()
    return res if res in ["en", "fr", "es", "de", "it", "pt"] else "en"

# ---------- Translation ----------
_tok, _mdl = None, None
def build_translator():
    global _tok, _mdl
    if _tok is None or _mdl is None:
        model = "facebook/m2m100_418M"
        _tok = AutoTokenizer.from_pretrained(model)
        _mdl = AutoModelForSeq2SeqLM.from_pretrained(model)
    return _tok, _mdl

def translate_line(text, tok, mdl, src_lang, tgt_lang="en", max_new_tokens=256):
    if not text.strip():
        return ""
    tok.src_lang = src_lang
    enc = tok(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        gen = mdl.generate(**enc, forced_bos_token_id=tok.get_lang_id(tgt_lang),
                           max_new_tokens=max_new_tokens)
    return tok.batch_decode(gen, skip_special_tokens=True)[0]

# ---------- Rendering back to page (layout-preserving) ----------
def draw_translations(base_img: Image.Image, translated_lines, font_path=None, font_size=FONT_SIZE, whitebox=WHITEBOX):
    img = base_img.copy()
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(font_path, font_size) if font_path else ImageFont.load_default()
    for text, (x, y, w, h) in translated_lines:
        if whitebox:
            draw.rectangle([x, y, x + w, y + h], fill="white")
        # naive wrapping if the translated text is much longer
        # (simple split by ~w/avg_char_width heuristic)
        avg_char_w = max(6, min(14, int(font_size * 0.6)))
        max_chars = max(1, int(w / avg_char_w))
        chunks = re.findall(rf'.{{1,{max_chars}}}(?:\s+|$)', text) if max_chars > 5 else [text]
        line_y = y
        for ch in chunks:
            draw.text((x, line_y), ch.strip(), font=font, fill="black")
            line_y += int(font_size * 1.2)
            if line_y > y + h:  # if overflow, we just let it spill slightly (simple approach)
                pass
    return img

# ---------- Save PDF from images ----------
def images_to_pdf(images):
    buf = io.BytesIO()
    if not images:
        return b""
    images[0].save(buf, format="PDF", save_all=True, append_images=[im.convert("RGB") for im in images[1:]])
    buf.seek(0)
    return buf.getvalue()

# ---------- Main pipeline (layout-preserving) ----------
def translate_document_preserving_layout(src_path: str, out_path: str = OUT_PATH):
    # 1) Load pages as images
    pages = load_image_or_pdf_pages(src_path, dpi=OCR_DPI)

    # 2) OCR lines on first page to guess language
    sample_lines = ocr_words_with_boxes(pages[0])
    sample_text = " ".join([t for t, _ in sample_lines])[:1500]
    src_lang = detect_lang(sample_text)
    print(f"Detected language: {src_lang}")

    # 3) Translator
    tok, mdl = build_translator()

    # 4) Page loop
    out_images = []
    for idx, page_img in enumerate(pages, 1):
        lines = ocr_words_with_boxes(page_img)
        translated_lines = []
        for text, bbox in lines:
            en = translate_line(text, tok, mdl, src_lang=src_lang, tgt_lang="en")
            translated_lines.append((en, bbox))
        rendered = draw_translations(page_img, translated_lines, font_size=FONT_SIZE, whitebox=WHITEBOX)
        out_images.append(rendered.convert("RGB"))
        print(f"Page {idx}: {len(lines)} lines translated")

    # 5) Save PDF
    pdf_bytes = images_to_pdf(out_images)
    with open(out_path, "wb") as f:
        f.write(pdf_bytes)
    print(f"✅ Layout-preserved translated PDF saved: {out_path}")

# ---------- Run ----------
if __name__ == "__main__":
    SRC_PATH = "/content/French - SO.pdf"   # <-- set your file path (PDF or image)
    translate_document_preserving_layout(SRC_PATH, out_path=OUT_PATH)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

Page 1: 19 lines translated
✅ Layout-preserved translated PDF saved: Translated_EN_PreservedLayout.pdf
